In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [15]:
outlet_1 = pd.read_excel('Master Data (Purchase).xlsx', sheet_name=0, parse_dates = ['Date of Purchase'])
outlet_2 = pd.read_excel('Master Data (Purchase).xlsx', sheet_name=1, parse_dates = ['Date of Purchase'])
outlet_3 = pd.read_excel('Master Data (Purchase).xlsx', sheet_name=2, parse_dates = ['Date of Purchase'])

In [16]:
def preprocessing(df, ingredient):
    df = df.copy()
    df.drop(['Product Name - Chinese', 'Product Name - English', 'Quantity of Order Received'], axis=1, inplace=True)
    tmp = df[df['SKU']==ingredient]
    tmp.set_index('Date of Purchase', inplace=True)
    tmp.drop('SKU', axis=1, inplace=True)
    idx = pd.date_range('2018-01-01', '2020-12-31')
    tmp = tmp.reindex(idx, fill_value=0)
    return tmp

In [17]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [18]:
def main_func(df, product):
    df = preprocessing(df, product)
    X, y = create_features(df, label='Quantity of Order Placed')
    features_and_target = pd.concat([X, y], axis=1)
    split_date = '2020-11-30'
    df_train = df.loc[df.index <= split_date].copy()
    df_test = df.loc[df.index > split_date].copy()
    model = Prophet()
    model.fit(df_train.reset_index() \
              .rename(columns={'index':'ds',
                               'Quantity of Order Placed':'y'}))
    df_test_fcst = model.predict(df=df_test.reset_index() \
                                   .rename(columns={'index':'ds'}))
    return df_test_fcst

In [19]:
def final_function(results):
    results = results[['ds', 'yhat']]
    results['ds'] = results.ds.dt.strftime('%Y/%m/%d')
    results['yhat'] = results['yhat'].round(1)
    results.columns = ['date', 'purchase']
    results = results.head(31)
    return results

In [20]:
product_ls = outlet_1['SKU'].unique()

In [21]:
outlet_ls = [outlet_1, outlet_2, outlet_3]

In [22]:
ans_1 = pd.read_excel('Answer Sheet.xlsx', sheet_name=1)
ans_2 = pd.read_excel('Answer Sheet.xlsx', sheet_name=2)
ans_3 = pd.read_excel('Answer Sheet.xlsx', sheet_name=3)
ans_ls = [ans_1, ans_2, ans_3]

In [23]:
#writer = pd.ExcelWriter('sales_predict.xlsx', engine='openpyxl')
for x in range(len(outlet_ls)):
    outlet=[]
    for y in range(len(product_ls)):
        results = main_func(outlet_ls[x], product_ls[y])
        results = final_function(results)
        results['purchase'] = results['purchase'].clip(lower=0)
        if(y==0):
            outlet = results
        else:
            outlet = outlet.append(results, ignore_index=True)
        print(str(outlet_ls[x]), product_ls[y], 'done!')
        print(outlet)
    ans_ls[x]['Quantity of Order Placed'] = outlet['purchase'] 

C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

       SKU Product Name - English Product Name - Chinese Date of Purchase  \
0     A001         Enoki Mushroom                    金针菇       2018-01-01   
1     A001         Enoki Mushroom                    金针菇       2018-01-02   
2     A001         Enoki Mushroom                    金针菇       2018-01-03   
3     A001         Enoki Mushroom                    金针菇       2018-01-04   
4     A001         Enoki Mushroom                    金针菇       2018-01-05   
...    ...                    ...                    ...              ...   
9980  A012           Baby Cabbage                    娃娃菜       2020-11-24   
9981  A012           Baby Cabbage                    娃娃菜       2020-11-25   
9982  A012           Baby Cabbage                    娃娃菜       2020-11-27   
9983  A012           Baby Cabbage                    娃娃菜       2020-11-28   
9984  A012           Baby Cabbage                    娃娃菜       2020-11-29   

      Quantity of Order Placed  Quantity of Order Received  
0             

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

       SKU Product Name - English Product Name - Chinese Date of Purchase  \
0     A001         Enoki Mushroom                    金针菇       2018-01-01   
1     A001         Enoki Mushroom                    金针菇       2018-01-02   
2     A001         Enoki Mushroom                    金针菇       2018-01-03   
3     A001         Enoki Mushroom                    金针菇       2018-01-04   
4     A001         Enoki Mushroom                    金针菇       2018-01-05   
...    ...                    ...                    ...              ...   
9980  A012           Baby Cabbage                    娃娃菜       2020-11-24   
9981  A012           Baby Cabbage                    娃娃菜       2020-11-25   
9982  A012           Baby Cabbage                    娃娃菜       2020-11-27   
9983  A012           Baby Cabbage                    娃娃菜       2020-11-28   
9984  A012           Baby Cabbage                    娃娃菜       2020-11-29   

      Quantity of Order Placed  Quantity of Order Received  
0             

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

...

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

       SKU Product Name - English Product Name - Chinese Date of Purchase  \
0     A001         Enoki Mushroom                    金针菇       2018-04-15   
1     A001         Enoki Mushroom                    金针菇       2018-04-16   
2     A001         Enoki Mushroom                    金针菇       2018-04-17   
3     A001         Enoki Mushroom                    金针菇       2018-04-18   
4     A001         Enoki Mushroom                    金针菇       2018-04-19   
...    ...                    ...                    ...              ...   
6638  A012           Baby Cabbage                    娃娃菜       2020-11-24   
6639  A012           Baby Cabbage                    娃娃菜       2020-11-26   
6640  A012           Baby Cabbage                    娃娃菜       2020-11-27   
6641  A012           Baby Cabbage                    娃娃菜       2020-11-28   
6642  A012           Baby Cabbage                    娃娃菜       2020-11-30   

      Quantity of Order Placed  Quantity of Order Received  
0             

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

       SKU Product Name - English Product Name - Chinese Date of Purchase  \
0     A001         Enoki Mushroom                    金针菇       2018-04-15   
1     A001         Enoki Mushroom                    金针菇       2018-04-16   
2     A001         Enoki Mushroom                    金针菇       2018-04-17   
3     A001         Enoki Mushroom                    金针菇       2018-04-18   
4     A001         Enoki Mushroom                    金针菇       2018-04-19   
...    ...                    ...                    ...              ...   
6638  A012           Baby Cabbage                    娃娃菜       2020-11-24   
6639  A012           Baby Cabbage                    娃娃菜       2020-11-26   
6640  A012           Baby Cabbage                    娃娃菜       2020-11-27   
6641  A012           Baby Cabbage                    娃娃菜       2020-11-28   
6642  A012           Baby Cabbage                    娃娃菜       2020-11-30   

      Quantity of Order Placed  Quantity of Order Received  
0             

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
# ans_1['Date of Purchase'] = ans_1['Date of Purchase'].dt.strftime('%Y/%m/%d')
# ans_2['Date of Purchase'] = ans_2['Date of Purchase'].dt.strftime('%Y/%m/%d')
# ans_3['Date of Purchase'] = ans_3['Date of Purchase'].dt.strftime('%Y/%m/%d')

In [25]:
writer = pd.ExcelWriter('Answer Sheet.xlsx', engine='openpyxl')
ans_1.to_excel(writer, sheet_name="Purchase - Outlet 1 (DEC)", index = False)
ans_2.to_excel(writer, sheet_name="Purchase - Outlet 2 (DEC)", index = False)
ans_3.to_excel(writer, sheet_name="Purchase - Outlet 3 (DEC)", index = False)
writer.save()

In [12]:
ans_1

,SKU,Product Name - English,Date of Purchase,Quantity of Order Placed
0,A001,Enoki Mushroom,2020/12/01,21.8
1,A001,Enoki Mushroom,2020/12/02,25.7
2,A001,Enoki Mushroom,2020/12/03,30.4
3,A001,Enoki Mushroom,2020/12/04,61.2
4,A001,Enoki Mushroom,2020/12/05,61.3
...,...,...,...,...
367,A012,Baby Cabbage,2020/12/27,4.4
368,A012,Baby Cabbage,2020/12/28,3.4
369,A012,Baby Cabbage,2020/12/29,2.1
370,A012,Baby Cabbage,2020/12/30,2.4


In [129]:
ans_1[ans_1['SKU']=='A001']

,SKU,Product Name - English,Date of Purchase,Quantity of Order Placed
0,A001,Enoki Mushroom,2020/12/01,22.0
1,A001,Enoki Mushroom,2020/12/02,26.0
2,A001,Enoki Mushroom,2020/12/03,30.0
3,A001,Enoki Mushroom,2020/12/04,61.0
4,A001,Enoki Mushroom,2020/12/05,61.0
5,A001,Enoki Mushroom,2020/12/06,32.0
6,A001,Enoki Mushroom,2020/12/07,24.0
7,A001,Enoki Mushroom,2020/12/08,27.0
8,A001,Enoki Mushroom,2020/12/09,32.0
9,A001,Enoki Mushroom,2020/12/10,37.0


In [116]:
results = main_func(outlet_ls[0], product_ls[0])
results = final_function(results)

C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [117]:
results.head(60)

,date,purchase
0,2020/11/01,29.0
1,2020/11/02,20.0
2,2020/11/03,22.0
3,2020/11/04,26.0
4,2020/11/05,30.0
5,2020/11/06,61.0
6,2020/11/07,60.0
7,2020/11/08,29.0
8,2020/11/09,21.0
9,2020/11/10,23.0
